**기본 세팅**

In [1]:
import numpy as np
import pandas as pd

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

import warnings

In [2]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

mpl.rc('font', family='NanumGothic') # 폰트 설정
mpl.rc('axes', unicode_minus=False) # 유니코드에서 음수 부호 설정

# 차트 스타일 설정
sns.set(font="NanumGothic", rc={"axes.unicode_minus":False}, style='darkgrid')
plt.rc("figure", figsize=(10,8))

warnings.filterwarnings("ignore")

---

# 10. 모델 설계하기

이번 챕터에서는 폐암 수술 환자 데이터를 이용해서 생존율을 예측할 것이다.

딥러닝을 이용해서 진행할 것이고 각 코드가 어떤 역할을 하는지 알아보자.

## 10.1 데이터 불러오기

In [3]:
# 폐암 수술 환자 데이터
Data_set = np.loadtxt("deeplearning/dataset/ThoraricSurgery.csv", delimiter = ",")
Data_set

array([[293.  ,   1.  ,   3.8 , ...,   0.  ,  62.  ,   0.  ],
       [  1.  ,   2.  ,   2.88, ...,   0.  ,  60.  ,   0.  ],
       [  8.  ,   2.  ,   3.19, ...,   0.  ,  66.  ,   1.  ],
       ...,
       [406.  ,   6.  ,   5.36, ...,   0.  ,  62.  ,   0.  ],
       [ 25.  ,   8.  ,   4.32, ...,   0.  ,  58.  ,   1.  ],
       [447.  ,   8.  ,   5.2 , ...,   0.  ,  49.  ,   0.  ]])

- 우선 csv파일로 된 데이터를 불러왔다.


- 그동안 보통 `pd.read_csv()`를 사용했었는데 Numpy로도 파일을 불러올 수 있었다.


- 데이터는 값만 있고 피처명은 없으나 마지막 컬럼이 생존 혹은 사망을 나타내는 타겟이다.

In [4]:
Data_set.shape

(470, 18)

- 총 470개의 샘플이 17개의 피처, 1개의 타겟으로 이루어져 있다.

In [5]:
# 피처, 타겟 분리
X = Data_set[:,:17]
Y = Data_set[:,17]

- 피처와 타겟은 따로 분리해두자.

## 10.2 모델 설정

In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import tensorflow as tf

# 시드 설정
np.random.seed(3)
tf.random.set_seed(3)

# 딥러닝 구조 결정
model = Sequential()
model.add(Dense(30, input_dim = 17, activation="relu")) # 입력 값으로 17개의 피처 모두 사용
model.add(Dense(1, activation="sigmoid"))

- 케라스에서는 `Sequential()`을 이용해 쉽게 층을 생성할 수 있다.


- `Sequential()`의 `add()`를 사용하면 새로운 층을 하나씩 추가한다.


- 각 층의 구체적인 구조는 `Dense()`를 이용해 결정한다.


- 맨 마지막 층은 출력층이 되며 나머지는 모두 은닉층 역할을 담당한다.


- 입력층은 따로 생성하지 않고 첫 번째 `Dense()`에 `input_dim`으로 입력 값의 개수를 정해준다.


- 첫 번째 은닉층의 `Dense()`는 17개의 입력 값을 받아 30개의 노드를 생성하며 활성화 함수로 렐루 함수를 사용한다.


- 두 번째 출력층의 `Dense()`는 1개의 노드를 생성하며 활성화 함수로 시그모이드 함수를 사용한다.

## 10.3 모델 컴파일

In [7]:
model.compile(loss="mean_squared_error", optimizer="adam", metrics=["accuracy"])

- `compile()`은 앞서 설정한 모델이 효과적으로 구현되게끔 여러가지 환경을 설정한다.


- 오차 함수, 최적화 방법, 모델 수행 결과 평가 지표 등을 설정 가능하다.

## 10.4 오차 함수

앞서 컴파일에 사용할 수 있는 오차 함수의 종류에 대해 조금 더 살펴보자.

현재는 MSE를 사용하였지만 교차 엔트로피 계열의 함수 등 다양한 오차 함수가 있다.

오차 함수를 바꿔줌에 따라 성능이 더 좋아지기도 한다.

지금 데이터는 생존/사망 이진 분류 문제인데 이런 경우는 보통 이항 교차 엔트로피를 사용한다.

아래는 오차 함수에 대해 일부 요약해두었다.

- **mean_squared_error**: 평균 제곱 오차


- **mean_absolute_error**: 평균 절대 오차(실제 값과 예측 값 차이의 절댓값 평균)


- **mean_absolute_percentage_error**: 평균 절대 백분율 오차(절대값 오차를 절대값으로 나눈 후 평균)


- **mean_squared_logarithmic_error**: 평균 제곱 로그 오차(실제 값과 예측 값에 로그를 적용한 값의 차이를 제곱한 값의 평균)


- **categorical_crossentropy**: 범주형 교차 엔트로피(일반적인 분류)


- **binary_crossentropy**: 이항 교차 엔트로피(두 개의 클래스 중에서 예측할 때)

## 10.5 모델 실행

In [8]:
model.fit(X, Y, epochs=100, batch_size=10)

Train on 470 samples
Epoch 1/100
470/470 [==============================] - 1s 1ms/sample - loss: 0.1495 - accuracy: 0.8404
Epoch 2/100
470/470 [==============================] - 0s 126us/sample - loss: 0.1447 - accuracy: 0.8511
Epoch 3/100
470/470 [==============================] - 0s 121us/sample - loss: 0.1448 - accuracy: 0.8511
Epoch 4/100
470/470 [==============================] - 0s 125us/sample - loss: 0.1453 - accuracy: 0.8489
Epoch 5/100
470/470 [==============================] - 0s 120us/sample - loss: 0.1438 - accuracy: 0.8511
Epoch 6/100
470/470 [==============================] - 0s 126us/sample - loss: 0.1418 - accuracy: 0.8511
Epoch 7/100
470/470 [==============================] - 0s 132us/sample - loss: 0.1425 - accuracy: 0.8511
Epoch 8/100
470/470 [==============================] - 0s 124us/sample - loss: 0.1427 - accuracy: 0.8468
Epoch 9/100
470/470 [==============================] - 0s 123us/sample - loss: 0.1446 - accuracy: 0.8489
Epoch 10/100
470/470 [==============

470/470 [==============================] - 0s 252us/sample - loss: 0.1249 - accuracy: 0.8574
Epoch 78/100
470/470 [==============================] - 0s 247us/sample - loss: 0.1341 - accuracy: 0.8489
Epoch 79/100
470/470 [==============================] - ETA: 0s - loss: 0.1322 - accuracy: 0.85 - 0s 236us/sample - loss: 0.1318 - accuracy: 0.8511
Epoch 80/100
470/470 [==============================] - 0s 302us/sample - loss: 0.1316 - accuracy: 0.8468
Epoch 81/100
470/470 [==============================] - 0s 259us/sample - loss: 0.1210 - accuracy: 0.8574
Epoch 82/100
470/470 [==============================] - 0s 218us/sample - loss: 0.1229 - accuracy: 0.8553
Epoch 83/100
470/470 [==============================] - 0s 242us/sample - loss: 0.1234 - accuracy: 0.8553
Epoch 84/100
470/470 [==============================] - 0s 238us/sample - loss: 0.1236 - accuracy: 0.8468
Epoch 85/100
470/470 [==============================] - 0s 225us/sample - loss: 0.1277 - accuracy: 0.8489
Epoch 86/100
470/

- 마지막으로 `fit()`을 이용해 모델을 실행할 수 있다.


- `epochs`는 지정한 숫자 만큼 샘플이 재사용 될 때 까지 반복하라는 뜻이다.


- `batch_size`는 한번에 사용할 샘플 수를 정한다.


- 즉 한 epoch 마다 모든 샘플을 사용하긴 하지만 batch_size 만큼 나누어서 집어 넣는다.


- [딥러닝의 동작 원리[로지스틱회귀]](https://romg2.github.io/dl_all/02-%EB%AA%A8%EB%91%90%EC%9D%98-%EB%94%A5%EB%9F%AC%EB%8B%9D-02.-%EB%94%A5%EB%9F%AC%EB%8B%9D%EC%9D%98-%EB%8F%99%EC%9E%91-%EC%9B%90%EB%A6%AC-(2)/)에서 경사하강법 코드로 예를 들면 교재는 batch_size를 1 나는 470, epoch는 동일하게 2001로 설정하였다.